# Estimating Non-Mandatory Tour Frequency



# Load libraries

In [1]:
import larch  # !conda install larch #for estimation
import larch.util.activitysim
import pandas as pd
import numpy as np
import yaml 
import larch.util.excel
import os

# Required Inputs

In addition to a working ActivitySim model setup, estimation mode requires an ActivitySim format household travel survey.  An ActivitySim format household travel survey is very similar to ActivitySim's simulation model tables:

 - households
 - persons
 - tours
 - joint_tour_participants
 - trips (not yet implemented)

Examples of the ActivitySim format household travel survey are included in the [example_estimation data folders](https://github.com/RSGInc/activitysim/tree/develop/activitysim/examples/example_estimation).  The user is responsible for formatting their household travel survey into the appropriate format.  

After creating an ActivitySim format household travel survey, the `scripts/infer.py` script is run to append additional calculated fields.  An example of an additional calculated field is the `household:joint_tour_frequency`, which is calculated based on the `tours` and `joint_tour_participants` tables.  

The input survey files are below.

### Survey households

In [2]:
pd.read_csv("../data_sf/survey_data/override_households.csv")

,household_id,TAZ,income,hhsize,HHT,auto_ownership,num_workers,joint_tour_frequency
0,2223759,16,144100,2,1,0,2,1_Main
1,990869,134,48000,2,1,2,2,0_tours
2,125886,113,25900,1,4,1,1,0_tours
3,727893,8,26100,2,1,0,1,0_tours
4,2741769,150,121600,4,1,2,1,0_tours
...,...,...,...,...,...,...,...,...
1995,663493,110,19180,1,6,1,1,0_tours
1996,569375,20,7400,1,6,1,0,0_tours
1997,1445193,17,75000,1,4,0,1,0_tours
1998,2833455,69,0,1,0,0,0,0_tours


### Survey persons

In [3]:
pd.read_csv("../data_sf/survey_data/override_persons.csv")

,person_id,household_id,age,PNUM,sex,pemploy,pstudent,ptype,school_taz,workplace_taz,free_parking_at_work,cdap_activity,mandatory_tour_frequency,_escort,_shopping,_othmaint,_othdiscr,_eatout,_social,non_mandatory_tour_frequency
0,166,166,54,1,2,3,3,4,-1,-1,False,N,NaN,0,0,0,0,1,0,4
1,197,197,46,1,2,3,3,4,-1,-1,False,N,NaN,0,1,0,0,0,0,16
2,268,268,46,1,1,3,3,4,-1,-1,False,N,NaN,0,0,1,1,0,0,9
3,375,375,54,1,2,3,3,4,-1,-1,False,N,NaN,0,0,1,0,0,0,8
4,387,387,44,1,2,3,3,4,-1,-1,False,N,NaN,1,0,0,1,0,0,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4401,7554799,2863464,93,1,2,3,3,5,-1,-1,False,N,NaN,0,0,0,1,0,0,1
4402,7554818,2863483,68,1,1,3,3,5,-1,-1,False,N,NaN,0,0,1,1,0,0,9
4403,7555141,2863806,93,1,2,3,3,5,-1,-1,False,N,NaN,0,2,0,1,0,0,17
4404,7555853,2864518,71,1,1,3,3,5,-1,-1,False,N,NaN,0,0,0,0,0,1,2


### Survey tours

In [4]:
pd.read_csv("../data_sf/survey_data/override_tours.csv")

,tour_id,survey_tour_id,person_id,household_id,tour_type,tour_category,destination,origin,start,end,tour_mode,survey_parent_tour_id,parent_tour_id,composition,tdd,atwork_subtour_frequency
0,25820,258200,629,629,school,mandatory,133.0,131.0,12.0,15.0,WALK,NaN,NaN,NaN,115,NaN
1,52265,522650,1274,1274,school,mandatory,188.0,166.0,9.0,15.0,WALK_LOC,NaN,NaN,NaN,76,NaN
2,1117937,11179370,27266,27266,school,mandatory,133.0,9.0,17.0,18.0,WALK_HVY,NaN,NaN,NaN,163,NaN
3,1148523,11485230,28012,28012,school,mandatory,12.0,10.0,17.0,22.0,WALK_LRF,NaN,NaN,NaN,167,NaN
4,1208547,12085470,29476,29476,school,mandatory,13.0,16.0,8.0,15.0,WALK_LOC,NaN,NaN,NaN,61,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5768,302942627,3029426270,7388844,2750003,maint,atwork,5.0,7.0,14.0,14.0,WALK,3.029426e+09,302942643.0,NaN,135,NaN
5769,305120465,3051204650,7441962,2758909,maint,atwork,110.0,2.0,12.0,13.0,SHARED2FREE,3.051205e+09,305120481.0,NaN,113,NaN
5770,308000655,3080006550,7512211,2820876,eat,atwork,14.0,1.0,12.0,13.0,WALK,3.080007e+09,308000690.0,NaN,113,NaN
5771,308073840,3080738400,7513996,2822661,eat,atwork,69.0,107.0,8.0,16.0,SHARED3FREE,3.080739e+09,308073875.0,NaN,62,NaN


### Survey joint tour participants

In [5]:
pd.read_csv("../data_sf/survey_data/survey_joint_tour_participants.csv")

,participant_id,tour_id,household_id,person_id,participant_num
0,22095828301,220958283,2223759,5389226,1
1,22095828302,220958283,2223759,5389227,2
2,14429508701,144295087,1606646,3519392,1
3,14429508702,144295087,1606646,3519393,2
4,28367651801,283676518,2628704,6918939,1
...,...,...,...,...,...
226,16297928102,162979281,1769918,3975105,2
227,16297928103,162979281,1769918,3975106,3
228,16297928104,162979281,1769918,3975107,4
229,26353054902,263530549,2519358,6427575,1


# Example Setup if Needed

To avoid duplication of inputs, especially model settings and expressions, the `example_estimation` depends on the `example`.  The following commands create an example setup for use.  The location of these example setups (i.e. the folders) are important because the paths are referenced in this notebook.  The commands below download the skims.omx for the SF county example from the [activitysim resources repository](https://github.com/RSGInc/activitysim_resources).

In [6]:
# !activitysim create -e example_estimation_sf -d test

# Run the Estimation Example

In [7]:
# %cd test
# !activitysim run -c configs_estimation/configs -c configs -o output -d data_sf

os.chdir(os.path.expanduser("~/sandbox/cdap-est/test_example_estimation"))

In [8]:
# Load the settings file from the original config directory, not the EDB

settings = yaml.load(
    open(os.path.join("configs","non_mandatory_tour_frequency.yaml"),"r"), 
    Loader=yaml.SafeLoader,
)
settings

{'SEGMENT_COL': 'ptype',
 'SPEC': 'non_mandatory_tour_frequency.csv',
 'SPEC_SEGMENTS': [{'NAME': 'PTYPE_FULL',
   'PTYPE': 1,
   'COEFFICIENTS': 'non_mandatory_tour_frequency_coeffs_PTYPE_FULL.csv'},
  {'NAME': 'PTYPE_PART',
   'PTYPE': 2,
   'COEFFICIENTS': 'non_mandatory_tour_frequency_coeffs_PTYPE_PART.csv'},
  {'NAME': 'PTYPE_UNIVERSITY',
   'PTYPE': 3,
   'COEFFICIENTS': 'non_mandatory_tour_frequency_coeffs_PTYPE_UNIVERSITY.csv'},
  {'NAME': 'PTYPE_NONWORK',
   'PTYPE': 4,
   'COEFFICIENTS': 'non_mandatory_tour_frequency_coeffs_PTYPE_NONWORK.csv'},
  {'NAME': 'PTYPE_RETIRED',
   'PTYPE': 5,
   'COEFFICIENTS': 'non_mandatory_tour_frequency_coeffs_PTYPE_RETIRED.csv'},
  {'NAME': 'PTYPE_DRIVING',
   'PTYPE': 6,
   'COEFFICIENTS': 'non_mandatory_tour_frequency_coeffs_PTYPE_DRIVING.csv'},
  {'NAME': 'PTYPE_SCHOOL',
   'PTYPE': 7,
   'COEFFICIENTS': 'non_mandatory_tour_frequency_coeffs_PTYPE_SCHOOL.csv'},
  {'NAME': 'PTYPE_PRESCHOOL',
   'PTYPE': 8,
   'COEFFICIENTS': 'non_mandatory_to

# Read EDB

The next step is to read the EDB, including the coefficients, model settings, utilities specification, and chooser and alternative data.

In [9]:
segment_names = [s['NAME'] for s in settings['SPEC_SEGMENTS']]
segment_names

['PTYPE_FULL',
 'PTYPE_PART',
 'PTYPE_UNIVERSITY',
 'PTYPE_NONWORK',
 'PTYPE_RETIRED',
 'PTYPE_DRIVING',
 'PTYPE_SCHOOL',
 'PTYPE_PRESCHOOL']

In [10]:
edb_directory = f"output/estimation_data_bundle/non_mandatory_tour_frequency"

def read_csv(filename, **kwargs):
    return pd.read_csv(os.path.join(edb_directory, filename), **kwargs)

In [11]:
spec = read_csv(f"non_mandatory_tour_frequency_SPEC.csv")
alt_def = read_csv(f"non_mandatory_tour_frequency_alternatives.csv", index_col=0)

In [12]:
coefficients = {}
chooser_data = {}
alt_values = {}

for segment_name in segment_names:
    coefficients[segment_name] = read_csv(
        f"{segment_name}/non_mandatory_tour_frequency_{segment_name}_coefficients.csv", 
        index_col='coefficient_name',
    )
    chooser_data[segment_name] = read_csv(
        f"{segment_name}/non_mandatory_tour_frequency_choosers_combined.csv",
    )
    alt_values[segment_name] = read_csv(
        f"{segment_name}/non_mandatory_tour_frequency_interaction_expression_values.csv", 
    )

In [13]:
# Assume all coefficients with exactly equal current values are
# actually the same estimated coefficient value and should be 
# treated as such by Larch.  Comment out this entire cell to relax
# this assumption, although be careful about the number of unique
# parameters to estimate in these models.

relabel_coef = {}
for segment_name in segment_names:
    coef_backwards_map = dict([(j,i) for i,j in coefficients[segment_name]['value'].items()])
    relabel_coef[segment_name] = r = coefficients[segment_name]['value'].map(coef_backwards_map)
    spec[segment_name] = spec[segment_name].map(r)


In [14]:
for k,v in relabel_coef[segment_name].items():
    print(k,v)

coef_escorting_tour coef_escorting_tour
coef_discretionary_tour coef_discretionary_tour
coef_shopping_tour coef_urban_and_discretionary_tour
coef_maintenance_tour coef_maintenance_tour
coef_visiting_or_social_tour coef_visiting_or_social_tour
coef_eating_out_tour coef_urban_and_discretionary_tour
coef_total_number_of_tours_is_0_no_prior_tours coef_total_number_of_tours_is_6_plus
coef_total_number_of_tours_is_0_prior_tours coef_urban_and_discretionary_tour
coef_total_number_of_tours_is_1 coef_total_number_of_tours_is_1
coef_total_number_of_tours_is_2 coef_total_number_of_tours_is_2
coef_total_number_of_tours_is_3 coef_total_number_of_tours_is_3
coef_total_number_of_tours_is_4 coef_total_number_of_tours_is_4
coef_total_number_of_tours_is_5 coef_total_number_of_tours_is_6_plus
coef_total_number_of_tours_is_6_plus coef_total_number_of_tours_is_6_plus
coef_number_of_mandatory_tours_and_tour_frequency_is_0 coef_urban_and_discretionary_tour
coef_number_of_mandatory_tours_and_tour_frequency_is

### Utility specification

In [15]:
spec

,Label,Description,Expression,PTYPE_FULL,PTYPE_PART,PTYPE_UNIVERSITY,PTYPE_NONWORK,PTYPE_RETIRED,PTYPE_DRIVING,PTYPE_SCHOOL,PTYPE_PRESCHOOL
0,util_escorting_tour,Escorting Tour,escort,coef_urban_and_discretionary_tour,coef_urban_and_discretionary_tour,coef_transit_access_to_retail_and_discretionary,coef_urban_and_discretionary_tour,coef_urban_and_discretionary_tour,coef_urban_and_discretionary_tour,coef_urban_and_discretionary_tour,coef_escorting_tour
1,util_discretionary_tour,Discretionary Tour,othdiscr,coef_urban_and_discretionary_tour,coef_urban_and_discretionary_tour,coef_transit_access_to_retail_and_discretionary,coef_urban_and_discretionary_tour,coef_urban_and_discretionary_tour,coef_urban_and_discretionary_tour,coef_urban_and_discretionary_tour,coef_discretionary_tour
2,util_shopping_tour,Shopping Tour,shopping,coef_urban_and_discretionary_tour,coef_urban_and_discretionary_tour,coef_transit_access_to_retail_and_discretionary,coef_urban_and_discretionary_tour,coef_urban_and_discretionary_tour,coef_urban_and_discretionary_tour,coef_urban_and_discretionary_tour,coef_urban_and_discretionary_tour
3,util_maintenance_tour,Maintenance Tour,othmaint,coef_urban_and_discretionary_tour,coef_urban_and_discretionary_tour,coef_transit_access_to_retail_and_discretionary,coef_urban_and_discretionary_tour,coef_urban_and_discretionary_tour,coef_urban_and_discretionary_tour,coef_urban_and_discretionary_tour,coef_maintenance_tour
4,util_visiting_or_social_tour,Visiting/Social Tour,social,coef_urban_and_discretionary_tour,coef_urban_and_discretionary_tour,coef_transit_access_to_retail_and_discretionary,coef_urban_and_discretionary_tour,coef_urban_and_discretionary_tour,coef_urban_and_discretionary_tour,coef_urban_and_discretionary_tour,coef_visiting_or_social_tour
...,...,...,...,...,...,...,...,...,...,...,...
205,util_1_plus_maintenance_tours_constant,1+ Maintenance Tours Constant,othmaint >= 1,coef_1_plus_maintenance_tours_constant,coef_1_plus_maintenance_tours_constant,coef_1_plus_maintenance_tours_constant,coef_1_plus_maintenance_tours_constant,coef_1_plus_maintenance_tours_constant,coef_1_plus_maintenance_tours_constant,coef_1_plus_maintenance_tours_constant,coef_1_plus_maintenance_tours_constant
206,util_1_plus_eating_out_tours_constant,1+ Eating Out Tours Constant,eatout >= 1,coef_1_plus_eating_out_tours_constant,coef_1_plus_eating_out_tours_constant,coef_1_plus_eating_out_tours_constant,coef_1_plus_eating_out_tours_constant,coef_1_plus_eating_out_tours_constant,coef_1_plus_eating_out_tours_constant,coef_1_plus_eating_out_tours_constant,coef_1_plus_eating_out_tours_constant
207,util_1_plus_visting_tours_constant,1+ Visting Tours Constant,social >= 1,coef_1_plus_visting_tours_constant,coef_1_plus_visting_tours_constant,coef_1_plus_visting_tours_constant,coef_1_plus_visting_tours_constant,coef_1_plus_visting_tours_constant,coef_1_plus_visting_tours_constant,coef_1_plus_visting_tours_constant,coef_1_plus_visting_tours_constant
208,util_1_plus_other_discretionary_tours_constant,1+ Other Discretionary Tours Constant,othdiscr >= 1,coef_1_plus_other_discretionary_tours_constant,coef_1_plus_other_discretionary_tours_constant,coef_1_plus_other_discretionary_tours_constant,coef_1_plus_other_discretionary_tours_constant,coef_1_plus_other_discretionary_tours_constant,coef_1_plus_other_discretionary_tours_constant,coef_1_plus_other_discretionary_tours_constant,coef_1_plus_other_discretionary_tours_constant


### Alternatives data

In [16]:
alt_values['PTYPE_FULL']

,person_id,variable,0,1,2,3,4,5,6,7,...,86,87,88,89,90,91,92,93,94,95
0,107640,util_0_auto_household_and_escorting_tour,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2
1,107640,util_1_escort_tour_constant,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,107640,util_1_plus_eating_out_tours_constant,False,False,False,False,True,True,True,True,...,True,True,False,False,False,False,True,True,True,True
3,107640,util_1_plus_maintenance_tours_constant,False,False,False,False,False,False,False,False,...,False,False,True,True,True,True,True,True,True,True
4,107640,util_1_plus_other_discretionary_tours_constant,False,True,False,True,False,True,False,True,...,False,True,False,True,False,True,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233095,7514060,util_zero_car_ownership_and_tour_frequency_is_1,False,True,True,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
233096,7514060,util_zero_car_ownership_and_tour_frequency_is_2,False,False,False,True,False,True,True,False,...,False,False,False,False,False,False,False,False,False,False
233097,7514060,util_zero_car_ownership_and_tour_frequency_is_3,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
233098,7514060,util_zero_car_ownership_and_tour_frequency_is_4,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False


### Chooser data

In [17]:
chooser_data['PTYPE_FULL']

,person_id,model_choice,override_choice,household_id,age,PNUM,sex,pemploy,pstudent,ptype,...,high_income,no_cars,car_sufficiency,num_hh_joint_shop_tours,num_hh_joint_eatout_tours,num_hh_joint_maint_tours,num_hh_joint_social_tours,num_hh_joint_othdiscr_tours,has_mandatory_tour,has_joint_tour
0,107640,0,0,107640,40,1,1,1,3,1,...,False,True,-1,0,0,0,0,0,1,0
1,107647,0,0,107647,37,1,1,1,3,1,...,False,True,-1,0,0,0,0,0,1,0
2,107659,6,6,107659,27,1,1,1,3,1,...,False,True,-1,0,0,0,0,0,1,0
3,107671,4,4,107671,33,1,2,1,3,1,...,False,False,0,0,0,0,0,0,1,0
4,107735,0,0,107735,30,1,1,1,3,1,...,False,False,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1105,7513554,0,0,2822219,18,1,2,1,3,1,...,False,True,-1,0,0,0,0,0,1,0
1106,7513844,1,1,2822509,24,1,1,1,3,1,...,False,True,-1,0,0,0,0,0,1,0
1107,7513962,0,0,2822627,31,1,1,1,3,1,...,False,True,-1,0,0,0,0,0,1,0
1108,7513972,0,0,2822637,19,1,1,1,3,1,...,False,True,-1,0,0,0,0,0,1,0


# Data Processing and Estimation Setup

The next step is to transform the EDB for larch for model re-estimation.  

In [18]:
from larch import P, X

### Utility specifications

In [19]:
m = {}
for segment_name in segment_names:
    segment_model = m[segment_name] = larch.Model()
    segment_model.utility_ca = larch.util.activitysim.linear_utility_from_spec(
        spec, x_col='Label', p_col=segment_name, 
    )
    larch.util.activitysim.apply_coefficients(coefficients[segment_name], segment_model)
    segment_model.choice_co_code = 'override_choice'

### Attach Data

In [20]:
for segment_name in segment_names:
    x_co = chooser_data[segment_name].set_index('person_id').rename(columns={'TAZ':'HOMETAZ'})
    x_ca = larch.util.activitysim.cv_to_ca(
        alt_values[segment_name].set_index(['person_id', 'variable'])
    )
    d = larch.DataFrames(
        co=x_co,
        ca=x_ca,
        av=True,
    )
    m[segment_name].dataservice = d

# Estimate

With the model setup for estimation, the next step is to estimate the model coefficients.  Make sure to use a sufficiently large enough household sample and set of zones to avoid an over-specified model, which does not have a numerically stable likelihood maximizing solution.  Larch has two built-in estimation methods: BHHH and SLSQP.  BHHH is the default and typically runs faster, but does not follow constraints on parameters.  SLSQP is safer, but slower, and may need additional iterations.

In [21]:
for segment_name in segment_names:
    m[segment_name].estimate(method='SLSQP', options={'maxiter':1000})

req_data does not request avail_ca or avail_co but it is set and being provided


,value,initvalue,nullvalue,minimum,maximum,holdfast,note,best
coef_0_auto_household_and_escorting_tour,-2.000000,0.0,0.0,NaN,NaN,1,,-2.000000
coef_1_escort_tour_constant,-0.330828,0.0,0.0,NaN,NaN,0,,-0.330828
coef_1_plus_eating_out_tours_constant,-3.930917,0.0,0.0,NaN,NaN,0,,-3.930917
coef_1_plus_maintenance_tours_constant,35.887919,0.0,0.0,NaN,NaN,0,,35.887919
coef_1_plus_other_discretionary_tours_constant,-102.837689,0.0,0.0,NaN,NaN,0,,-102.837689
...,...,...,...,...,...,...,...,...
coef_walk_access_to_retail_and_discretionary,-1.916255,0.0,0.0,NaN,NaN,0,,-1.916255
coef_walk_access_to_retail_and_eating_out,-0.194407,0.0,0.0,NaN,NaN,0,,-0.194407
coef_walk_access_to_retail_and_escorting,-0.447801,0.0,0.0,NaN,NaN,0,,-0.447801
coef_walk_access_to_retail_and_shopping,0.997844,0.0,0.0,NaN,NaN,0,,0.997844


<ipython-input-21-48db69312ba3>:2: PossibleOverspecification: WARNING: Model is possibly over-specified (hessian is nearly singular).
  m[segment_name].estimate(method='SLSQP', options={'maxiter':1000})
/Users/jeffnewman/OneDrive - Cambridge Systematics/Git/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 2.7822957483806973e-18 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
<ipython-input-21-48db69312ba3>:2: RuntimeWarning: invalid value encountered in sqrt
  m[segment_name].estimate(method='SLSQP', options={'maxiter':1000})
req_data does not request avail_ca or avail_co but it is set and being provided


,value,initvalue,nullvalue,minimum,maximum,holdfast,note,best
coef_0_auto_household_and_escorting_tour,-2.000000,0.0,0.0,NaN,NaN,1,,-2.000000
coef_1_escort_tour_constant,0.673740,0.0,0.0,NaN,NaN,0,,0.673740
coef_1_plus_eating_out_tours_constant,1.417410,0.0,0.0,NaN,NaN,0,,1.417410
coef_1_plus_maintenance_tours_constant,2.636858,0.0,0.0,NaN,NaN,0,,2.636858
coef_1_plus_other_discretionary_tours_constant,2.672552,0.0,0.0,NaN,NaN,0,,2.672552
coef_1_plus_shopping_tours_constant,2.434402,0.0,0.0,NaN,NaN,0,,2.434402
coef_1_plus_visting_tours_constant,1.408868,0.0,0.0,NaN,NaN,0,,1.408868
coef_2_plus_escort_tours_constant,2.118856,0.0,0.0,NaN,NaN,0,,2.118856
coef_car_shortage_vs_workers_and_tour_frequency_is_5_plus,-1.040045,0.0,0.0,NaN,NaN,0,,-1.040045
coef_female_and_discretionary_tour,-0.243132,0.0,0.0,NaN,NaN,0,,-0.243132


<ipython-input-21-48db69312ba3>:2: PossibleOverspecification: WARNING: Model is possibly over-specified (hessian is nearly singular).
  m[segment_name].estimate(method='SLSQP', options={'maxiter':1000})
/Users/jeffnewman/OneDrive - Cambridge Systematics/Git/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 3.08433259318469e-08 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
<ipython-input-21-48db69312ba3>:2: RuntimeWarning: invalid value encountered in sqrt
  m[segment_name].estimate(method='SLSQP', options={'maxiter':1000})
req_data does not request avail_ca or avail_co but it is set and being provided


...

<ipython-input-21-48db69312ba3>:2: PossibleOverspecification: WARNING: Model is possibly over-specified (hessian is nearly singular).
  m[segment_name].estimate(method='SLSQP', options={'maxiter':1000})
/Users/jeffnewman/OneDrive - Cambridge Systematics/Git/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 1.4736180457568787e-66 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
<ipython-input-21-48db69312ba3>:2: RuntimeWarning: invalid value encountered in sqrt
  m[segment_name].estimate(method='SLSQP', options={'maxiter':1000})
req_data does not request avail_ca or avail_co but it is set and being provided


,value,initvalue,nullvalue,minimum,maximum,holdfast,note,best
coef_0_auto_household_and_escorting_tour,-2.000000,0.0,0.0,NaN,NaN,1,,-2.000000
coef_1_escort_tour_constant,19.965420,0.0,0.0,NaN,NaN,0,,19.965420
coef_1_plus_eating_out_tours_constant,32.817464,0.0,0.0,NaN,NaN,0,,32.817464
coef_1_plus_maintenance_tours_constant,25.143245,0.0,0.0,NaN,NaN,0,,25.143245
coef_1_plus_other_discretionary_tours_constant,26.495420,0.0,0.0,NaN,NaN,0,,26.495420
coef_1_plus_shopping_tours_constant,25.891431,0.0,0.0,NaN,NaN,0,,25.891431
coef_1_plus_visting_tours_constant,25.436484,0.0,0.0,NaN,NaN,0,,25.436484
coef_2_plus_escort_tours_constant,-272.759082,0.0,0.0,NaN,NaN,0,,-272.759082
coef_auto_access_to_retail_and_escorting,56.916529,0.0,0.0,NaN,NaN,0,,56.916529
coef_high_income_group_and_eating_out_tour,0.597269,0.0,0.0,NaN,NaN,0,,0.597269


<ipython-input-21-48db69312ba3>:2: PossibleOverspecification: WARNING: Model is possibly over-specified (hessian is nearly singular).
  m[segment_name].estimate(method='SLSQP', options={'maxiter':1000})
/Users/jeffnewman/OneDrive - Cambridge Systematics/Git/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 3.6201940176916516e-16 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
<ipython-input-21-48db69312ba3>:2: RuntimeWarning: invalid value encountered in sqrt
  m[segment_name].estimate(method='SLSQP', options={'maxiter':1000})
req_data does not request avail_ca or avail_co but it is set and being provided


,value,initvalue,nullvalue,minimum,maximum,holdfast,note,best
coef_0_auto_household_and_escorting_tour,-2.000000,0.0,0.0,NaN,NaN,1,,-2.000000
coef_1_escort_tour_constant,7.256691,0.0,0.0,NaN,NaN,0,,7.256691
coef_1_plus_eating_out_tours_constant,-0.553987,0.0,0.0,NaN,NaN,0,,-0.553987
coef_1_plus_maintenance_tours_constant,-0.267705,0.0,0.0,NaN,NaN,0,,-0.267705
coef_1_plus_other_discretionary_tours_constant,-0.315828,0.0,0.0,NaN,NaN,0,,-0.315828
coef_1_plus_shopping_tours_constant,-24.415685,0.0,0.0,NaN,NaN,0,,-24.415685
coef_1_plus_visting_tours_constant,-0.954742,0.0,0.0,NaN,NaN,0,,-0.954742
coef_2_plus_escort_tours_constant,-5.727773,0.0,0.0,NaN,NaN,0,,-5.727773
coef_discretionary_tour,-0.906328,0.0,0.0,NaN,NaN,0,,-0.906328
coef_escorting_tour,-6.513856,0.0,0.0,NaN,NaN,0,,-6.513856


<ipython-input-21-48db69312ba3>:2: PossibleOverspecification: WARNING: Model is possibly over-specified (hessian is nearly singular).
  m[segment_name].estimate(method='SLSQP', options={'maxiter':1000})
/Users/jeffnewman/OneDrive - Cambridge Systematics/Git/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 2.805222450065217e-10 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")


### Estimated coefficients

In [22]:
for segment_name in segment_names:
    display(m[segment_name].parameter_summary())

,Value,Std Err,t Stat,Signif,Like Ratio,Null Value,Constrained
coef_0_auto_household_and_escorting_tour,-2.00,NA,NA,,NA,0.00,fixed value
coef_1_escort_tour_constant,-0.331,690.,-0.00,,NA,0.00,
coef_1_plus_eating_out_tours_constant,-3.93,253.,-0.02,,NA,0.00,
coef_1_plus_maintenance_tours_constant,35.9,253.,0.14,,NA,0.00,
coef_1_plus_other_discretionary_tours_constant,-103.,254.,-0.41,,NA,0.00,
coef_1_plus_shopping_tours_constant,22.6,258.,0.09,,NA,0.00,
coef_1_plus_visting_tours_constant,-7.23,253.,-0.03,,NA,0.00,
coef_2_plus_escort_tours_constant,-0.154,1.38e+03,-0.00,,NA,0.00,
coef_at_home_pre_driving_school_kid_and_escorting_tour,-44.6,0.000465,-96021.04,***,NA,0.00,
coef_at_home_pre_school_kid_and_discretionary_tour,-0.462,1.20,-0.39,,NA,0.00,


,Value,Std Err,t Stat,Signif,Like Ratio,Null Value,Constrained
coef_0_auto_household_and_escorting_tour,-2.00,NA,NA,,NA,0.00,fixed value
coef_1_escort_tour_constant,0.674,NA,NA,[*],2.35,0.00,
coef_1_plus_eating_out_tours_constant,1.42,NA,NA,[***],17.10,0.00,
coef_1_plus_maintenance_tours_constant,2.64,NA,NA,[***],88.65,0.00,
coef_1_plus_other_discretionary_tours_constant,2.67,NA,NA,[***],95.71,0.00,
coef_1_plus_shopping_tours_constant,2.43,NA,NA,[***],88.16,0.00,
coef_1_plus_visting_tours_constant,1.41,NA,NA,[***],11.85,0.00,
coef_2_plus_escort_tours_constant,2.12,NA,NA,[***],8.45,0.00,
coef_car_shortage_vs_workers_and_tour_frequency_is_5_plus,-1.04,0.404,-2.57,*,NA,0.00,
coef_female_and_discretionary_tour,-0.243,0.323,-0.75,,NA,0.00,


,Value,Std Err,t Stat,Signif,Like Ratio,Null Value,Constrained
coef_0_auto_household_and_escorting_tour,-2.00,NA,NA,,NA,0.00,fixed value
coef_1_escort_tour_constant,1.36,NA,NA,[***],6.00,0.00,
coef_1_plus_eating_out_tours_constant,11.9,19.2,0.62,,NA,0.00,
coef_1_plus_maintenance_tours_constant,0.254,NA,NA,[],0.37,0.00,
coef_1_plus_other_discretionary_tours_constant,-26.3,1.41e+03,-0.02,,NA,0.00,
coef_1_plus_shopping_tours_constant,8.34,NA,NA,[***],273.44,0.00,
coef_1_plus_visting_tours_constant,0.987,NA,NA,[*],2.01,0.00,
coef_2_plus_escort_tours_constant,2.92,NA,NA,[***],7.43,0.00,
coef_auto_access_to_retail_and_discretionary,6.09,4.68,1.30,,NA,0.00,
coef_auto_access_to_retail_and_eating_out,-2.02,5.44,-0.37,,NA,0.00,


,Value,Std Err,t Stat,Signif,Like Ratio,Null Value,Constrained
coef_0_auto_household_and_escorting_tour,-2.00,NA,NA,,NA,0.00,fixed value
coef_1_escort_tour_constant,-7.44,NA,NA,[***],1366.78,0.00,
coef_1_plus_eating_out_tours_constant,-7.34,NA,NA,[***],1559.80,0.00,
coef_1_plus_maintenance_tours_constant,26.5,NA,NA,[***],2102.97,0.00,
coef_1_plus_other_discretionary_tours_constant,-8.99,NA,NA,[***],1847.10,0.00,
coef_1_plus_shopping_tours_constant,-7.02,NA,NA,[***],1211.54,0.00,
coef_1_plus_visting_tours_constant,-7.76,NA,NA,[***],1659.66,0.00,
coef_2_plus_escort_tours_constant,-13.7,NA,NA,[***],2588.03,0.00,
coef_at_home_pre_driving_school_kid_and_escorting_tour,0.123,0.769,0.16,,NA,0.00,
coef_at_home_pre_school_kid_and_eating_out_tour,0.251,0.643,0.39,,NA,0.00,


,Value,Std Err,t Stat,Signif,Like Ratio,Null Value,Constrained
coef_0_auto_household_and_escorting_tour,-2.00,NA,NA,,NA,0.00,fixed value
coef_1_escort_tour_constant,0.894,354.,0.00,,NA,0.00,
coef_1_plus_eating_out_tours_constant,1.12,354.,0.00,,NA,0.00,
coef_1_plus_maintenance_tours_constant,0.824,354.,0.00,,NA,0.00,
coef_1_plus_other_discretionary_tours_constant,1.41,354.,0.00,,NA,0.00,
coef_1_plus_shopping_tours_constant,1.55,354.,0.00,,NA,0.00,
coef_1_plus_visting_tours_constant,1.13,354.,0.00,,NA,0.00,
coef_2_plus_escort_tours_constant,1.29,708.,0.00,,NA,0.00,
coef_car_surplus_vs_workers_and_tour_frequency_is_1,0.404,2.02,0.20,,NA,0.00,
coef_car_surplus_vs_workers_and_tour_frequency_is_5_plus,1.09,2.05,0.53,,NA,0.00,


,Value,Std Err,t Stat,Signif,Like Ratio,Null Value,Constrained
coef_0_auto_household_and_escorting_tour,-153.,0.00,NA,[*],2.54,0.00,
coef_1_escort_tour_constant,-25.0,152.,-0.16,,NA,0.00,
coef_1_plus_eating_out_tours_constant,-109.,0.0623,-1751.38,***,NA,0.00,
coef_1_plus_maintenance_tours_constant,-13.7,403.,-0.03,,NA,0.00,
coef_1_plus_other_discretionary_tours_constant,-24.4,152.,-0.16,,NA,0.00,
coef_1_plus_shopping_tours_constant,-134.,2.02e+04,-0.01,,NA,0.00,
coef_1_plus_visting_tours_constant,-25.9,152.,-0.17,,NA,0.00,
coef_2_plus_escort_tours_constant,-7.58,4.73e+04,-0.00,,NA,0.00,
coef_auto_access_to_retail_and_tour_frequency_is_5_plus,4.14,15.7,0.26,,NA,0.00,
coef_car_shortage_vs_workers_and_tour_frequency_is_5_plus,66.1,100.,0.66,,NA,0.00,


,Value,Std Err,t Stat,Signif,Like Ratio,Null Value,Constrained
coef_0_auto_household_and_escorting_tour,-2.00,NA,NA,,NA,0.00,fixed value
coef_1_escort_tour_constant,20.0,NA,NA,[***],18.89,0.00,
coef_1_plus_eating_out_tours_constant,32.8,274.,0.12,,NA,0.00,
coef_1_plus_maintenance_tours_constant,25.1,274.,0.09,,NA,0.00,
coef_1_plus_other_discretionary_tours_constant,26.5,274.,0.10,,NA,0.00,
coef_1_plus_shopping_tours_constant,25.9,274.,0.09,,NA,0.00,
coef_1_plus_visting_tours_constant,25.4,274.,0.09,,NA,0.00,
coef_2_plus_escort_tours_constant,-273.,NA,NA,[],0.00,0.00,
coef_auto_access_to_retail_and_escorting,56.9,80.7,0.71,,NA,0.00,
coef_high_income_group_and_eating_out_tour,0.597,1.01,0.59,,NA,0.00,


,Value,Std Err,t Stat,Signif,Null Value,Constrained
coef_0_auto_household_and_escorting_tour,-2.00,NA,NA,,0.00,fixed value
coef_1_escort_tour_constant,7.26,741.,0.01,,0.00,
coef_1_plus_eating_out_tours_constant,-0.554,1.33e+03,-0.00,,0.00,
coef_1_plus_maintenance_tours_constant,-0.268,3.56e+03,-0.00,,0.00,
coef_1_plus_other_discretionary_tours_constant,-0.316,6.05e+03,-0.00,,0.00,
coef_1_plus_shopping_tours_constant,-24.4,7.05e+03,-0.00,,0.00,
coef_1_plus_visting_tours_constant,-0.955,2.14e+03,-0.00,,0.00,
coef_2_plus_escort_tours_constant,-5.73,3.17e+04,-0.00,,0.00,
coef_discretionary_tour,-0.906,6.54e+03,-0.00,,0.00,
coef_escorting_tour,-6.51,1.61e+03,-0.00,,0.00,


# Output Estimation Results

In [23]:
for segment_name in segment_names:
    est_names = [j for j in coefficients[segment_name].index if j in m[segment_name].pf.index]
    try:
        for unique_coef,common_coef in relabel_coef[segment_name].items():
            coefficients[segment_name].loc[unique_coef,'value'] = m[segment_name].pf.loc[common_coef, 'value']
    except NameError:
        coefficients[segment_name].loc[est_names,'value'] = m[segment_name].pf.loc[est_names, 'value']

In [24]:
os.makedirs(os.path.join(edb_directory,'estimated'), exist_ok=True)

### Write the re-estimated coefficients file

In [25]:
for segment_name in segment_names:
    coefficients[segment_name].reset_index().to_csv(
        os.path.join(
            edb_directory,
            'estimated',
            f"non_mandatory_tour_frequency_{segment_name}_coefficients.csv",
        ), 
        index=False,
    )

### Write the model estimation report, including coefficient t-statistic and log likelihood

In [26]:
for segment_name in segment_names:
    m[segment_name].to_xlsx(
        os.path.join(
            edb_directory,
            'estimated',
            f"non_mandatory_tour_frequency_{segment_name}_model_estimation.xlsx",
        ), 
        data_statistics=False
    )